# Prediction pipline 

Working directory 

In [2]:
#Change working directory to project direcgtory
import os

current_directory = os.getcwd()
project_name = 'bcppmchurn'
project_path = os.path.join(current_directory.split(project_name)[0], project_name)
os.chdir(project_path)
print(os.getcwd())  

/Users/hamza_hajjini/Desktop/DATA/HAMAZA_HAJJINI/projects/bcppmchurn


Packages 

In [26]:
import pandas as pd 
import numpy as np 

from src.components.data_ingestion import get_feature_tables_from_impala, get_target_table 
from src.components.data_structuring import StructuringPipeline
from src.components.data_processing import run_inference_data_processing_pipeline
from src.pipelines import prediction_pipeline
from src.pipelines.prediction_pipeline import PredictionPipeline
#from src.components.data_processing import 

In [11]:
import importlib
from src.components import data_structuring, data_ingestion, data_processing
importlib.reload(data_processing)
from src.components.data_structuring import StructuringPipeline
from src.components.data_ingestion import get_feature_tables_from_impala, get_target_table 
from src.components.data_processing import run_inference_data_processing_pipeline


Prediction pipeline

Ingestion and structuring 

In [ ]:
domains =["data", "voice", "complaints", "payement"]
feature_types = ["stat", "trend"]
dn_group_interval = ["00", "14"]
batch_date = "2024-11-01"
#Initiate a spark session and get table features 
features_dict = get_feature_tables_from_impala(domains, feature_types, dn_group_interval, objective="inference") 
#Get target table
churners_non_churners = get_target_table(table_name="tel_test_dtddds.dev_BCPPMCHURN_target_table_pc3_20241101")
#Structure table featues into a dataframe
df, pivoted_df = StructuringPipeline(features_dict, churners_non_churners, objective = "inference").run_structuring_pipeline(batch_date)
df.head()

In [4]:
df, pivoted_df = StructuringPipeline(features_dict, churners_non_churners, objective = "inference").run_structuring_pipeline(batch_date)
df.head()

merging and concatinating feature tables completed successfully
pivoting table
Joining pivoted table with target table
Joining pivoted table with target table
droping duplicated dns
Saving df


,dn,gamme,churn_segment,churn_date,activation_bscs_date,id_date,complaints_complaints_complaint_status_abondon_duration_nb_1m,complaints_complaints_complaint_status_abondon_total_nb_1m,complaints_complaints_complaint_status_autre_duration_nb_1m,complaints_complaints_complaint_status_autre_total_nb_1m,...,voice_call_direction_ic_duration_value_1_3w_3_4w,voice_call_direction_oc_duration_value_1_3w_3_4w,voice_destination_type_international_duration_value_1_3w_3_4w,voice_destination_type_national_duration_value_1_3w_3_4w,voice_termination_type_offnet_duration_value_1_3w_3_4w,voice_termination_type_onnet_duration_value_1_3w_3_4w,voice_weekend_n_duration_value_1_3w_3_4w,voice_weekend_y_duration_value_1_3w_3_4w,voice_workingh_n_duration_value_1_3w_3_4w,voice_workingh_y_duration_value_1_3w_3_4w
0,212662793240,Forfaits 99 dhs,non_churners,2024-11-01,2024-07-18 12:34:47,20241101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,212634650449,Forfaits 99 dhs,non_churners,2024-11-01,2021-01-29 16:46:53,20241101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,212630202340,Forfaits Hors 99 dhs,non_churners,2024-11-01,2017-02-14 14:04:11,20241101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,212714094566,Forfaits 49 dhs,non_churners,2024-11-01,2023-11-23 13:06:15,20241101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,212606501191,Forfaits 49 dhs,non_churners,2024-11-01,2022-11-18 11:50:45,20241101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df.shape

(1066478, 3631)

Delete all the rows where all the features are non (of course except target columns)

In [45]:
#Create a copy of df
df_test = df.copy()

In [34]:
#Reset index 
df_test = df_test.reset_index(drop=True)

In [47]:
THERESHOLD = df_test.shape[1] - churners_non_churners.shape[1]
index_to_be_deleted = df_test.index[df_test.isna().sum(axis=1) >= THERESHOLD].tolist()
print ("Number of index to be deleted:", len(index_to_be_deleted))

Number of index to be deleted: 897447


In [48]:
#Drop these indexes from df
df_test = df_test.drop(index = index_to_be_deleted)
df_test.shape

(169031, 3631)

In [49]:
df_test.head()

,dn,gamme,churn_segment,churn_date,activation_bscs_date,id_date,complaints_complaints_complaint_status_abondon_duration_nb_1m,complaints_complaints_complaint_status_abondon_total_nb_1m,complaints_complaints_complaint_status_autre_duration_nb_1m,complaints_complaints_complaint_status_autre_total_nb_1m,...,voice_call_direction_ic_duration_value_1_3w_3_4w,voice_call_direction_oc_duration_value_1_3w_3_4w,voice_destination_type_international_duration_value_1_3w_3_4w,voice_destination_type_national_duration_value_1_3w_3_4w,voice_termination_type_offnet_duration_value_1_3w_3_4w,voice_termination_type_onnet_duration_value_1_3w_3_4w,voice_weekend_n_duration_value_1_3w_3_4w,voice_weekend_y_duration_value_1_3w_3_4w,voice_workingh_n_duration_value_1_3w_3_4w,voice_workingh_y_duration_value_1_3w_3_4w
16,212603971703,Forfaits 49 dhs,non_churners,2024-11-01,2019-12-05 17:05:26,20241101,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,212707542311,Forfaits 49 dhs,non_churners,2024-11-01,2021-10-22 09:54:26,20241101,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,212608768406,Forfaits 49 dhs,non_churners,2024-11-01,2021-06-28 12:32:22,20241101,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,212615364402,Forfaits Hors 99 dhs,non_churners,2024-11-01,2019-08-20 21:38:23,20241101,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,212702254606,Forfaits Hors 99 dhs,non_churners,2024-11-01,2022-02-25 19:40:55,20241101,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Save dataframe
df_test.to_csv(f"data/experiments_data/{batch_date}_final_df.csv")

-----------------------

In [5]:
#OR LOAD DATA 
batch_date = "2024-11-01"
df = pd.read_csv(f"data/experiments_data/{batch_date}_final_df.csv")
df.shape 

(169031, 3632)

Data processing 

In [6]:
df.head()

,Unnamed: 0,dn,gamme,churn_segment,churn_date,activation_bscs_date,id_date,complaints_complaints_complaint_status_abondon_duration_nb_1m,complaints_complaints_complaint_status_abondon_total_nb_1m,complaints_complaints_complaint_status_autre_duration_nb_1m,...,voice_call_direction_ic_duration_value_1_3w_3_4w,voice_call_direction_oc_duration_value_1_3w_3_4w,voice_destination_type_international_duration_value_1_3w_3_4w,voice_destination_type_national_duration_value_1_3w_3_4w,voice_termination_type_offnet_duration_value_1_3w_3_4w,voice_termination_type_onnet_duration_value_1_3w_3_4w,voice_weekend_n_duration_value_1_3w_3_4w,voice_weekend_y_duration_value_1_3w_3_4w,voice_workingh_n_duration_value_1_3w_3_4w,voice_workingh_y_duration_value_1_3w_3_4w
0,16,212603971703,Forfaits 49 dhs,non_churners,2024-11-01,2019-12-05 17:05:26,20241101,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,212707542311,Forfaits 49 dhs,non_churners,2024-11-01,2021-10-22 09:54:26,20241101,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19,212608768406,Forfaits 49 dhs,non_churners,2024-11-01,2021-06-28 12:32:22,20241101,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20,212615364402,Forfaits Hors 99 dhs,non_churners,2024-11-01,2019-08-20 21:38:23,20241101,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,37,212702254606,Forfaits Hors 99 dhs,non_churners,2024-11-01,2022-02-25 19:40:55,20241101,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
importlib.reload(data_processing)
from src.components.data_processing import run_inference_data_processing_pipeline
df_norm, dns = run_inference_data_processing_pipeline(df, "2024-12-23")

Extracting dns list from df .......................................................
Computing customer tenur
Number of missing dates when caculating tenure is 0
Selecting inference featrues from df .......................................................
Filling nan values with 0 .......................................................
Total number of missing values in df_train after filling all nan with 0 is : 0 ............................................
Encoding gamme feature .......................................................
df columns Index(['complaints_complaints_complaint_status_resolu_duration_nb_1m',
       'complaints_complaints_complaint_status_resolu_total_nb_1m',
       'complaints_complaints_complaint_type_reclamation_total_nb_1m',
       'complaints_complaints_complaint_type_retention_total_nb_1m',
       'complaints_complaints_weekend_n_duration_nb_1m',
       'complaints_complaints_weekend_n_total_nb_1m',
       'complaints_complaints_workingh_y_duration_nb_1m',
  

Prediction pipeline 

In [ ]:
dns_churn = PredictionPipeline().run_prediction_pipeline("2024-12-23", "")